In [1]:
import tensorflow.keras as keras
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import inception_resnet_v2
import numpy as np
import os, glob, sys
from PIL import Image

## Load data and preprocess images

In [2]:
# converting all jpg pictures to png format
base_path = './data/'
classes = ['oak', 'maple', 'beech']
for leaf in classes:
    path = base_path + leaf
    for file_name in os.listdir(path):        
        if file_name.endswith('.jpg'):
            im = Image.open(f'{path}/{file_name}')
            last_char_index = file_name.rfind(".jpg")
            new_name = file_name[:last_char_index] + ".png"
            im.save(f'{path}/{new_name}')
            os.remove(os.path.join(path, file_name))

## Pipeline like data upload

In [3]:
# define an image data generator
data_gen_train = preprocessing.image.ImageDataGenerator(
    preprocessing_function=inception_resnet_v2.preprocess_input)

In [4]:
data_gen_test = preprocessing.image.ImageDataGenerator(
    preprocessing_function=inception_resnet_v2.preprocess_input)

In [5]:
# a generator that returns batches of X and y arrays
train_data_gen = data_gen_train.flow_from_directory(
        directory=base_path,
        class_mode="categorical",
        classes=classes,
        batch_size=1200,
        target_size=(224, 224))

Found 1159 images belonging to 3 classes.


In [6]:
test_data_gen = data_gen_test.flow_from_directory(
        directory=base_path,
        class_mode="categorical",
        classes=classes,
        batch_size=197,
        target_size=(224, 224))

Found 1159 images belonging to 3 classes.


In [7]:
X_train, y_train = next(train_data_gen)

In [8]:
X_test, y_test = next(test_data_gen)

## Model pipeline building on inception_resnet_v2

In [ ]:
base_model = keras.applications.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
    pooling='avg')
base_model.trainable = False

In [ ]:
model = keras.Sequential()
model.add(base_model)
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(len(classes), activation='softmax'))

In [ ]:
%%time
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, epochs=50, verbose=0, batch_size=300, callbacks=[callback], validation_split=0.2)

In [ ]:
%%time
base_model.trainable = True
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, epochs=10, verbose=0, batch_size=300, callbacks=[callback], validation_split=0.2)

In [15]:
model.save('maple_oak_beech_basemodel_trained.h5')